In [8]:
import json
import requests
import dlt

from dlt.destinations import qdrant

In [6]:
dlt.__version__

'1.13.0'

In [2]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course["course"]

        for doc in course["documents"]:
            doc["course"] = course_name
            yield doc

In [4]:
qdrant_destination = qdrant(
    qd_path="db.qdrant",
)

In [5]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data",
)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

d:\Project\llm-zoomcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]
d:\Project\llm-zoomcamp\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ariaw\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run

Run started at 2025-07-10 16:04:19.496318+00:00 and COMPLETED in 4 minutes and 39.48 seconds with 4 steps.
Step extract COMPLETED in 3.11 seconds.

Load package 1752163718.7670643 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.39 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752163718.7670643 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 16.72 seconds.
Pipeline zoomcamp_pipeline load step completed in 16.63 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used d:\Project\llm-zoomcamp\dlt\db.qdrant location to store data
Load package 1752163718.7670643 is LOADED and contains no failed jobs

Step run COMPLETED in 4 minutes and 39.48 seconds.
Pipeline zoomcamp_pipeline load step completed in 16.63 seconds
1 load package(

In [9]:
with open('db.qdrant/meta.json', 'r') as f:
    metadata = json.load(f)

In [17]:
metadata["collections"]["zoomcamp_tagged_data"]["vectors"].keys()

dict_keys(['fast-bge-small-en'])